In [6]:
!pip install transformers langchain datasets faiss-cpu
!pip install -U langchain-community
!pip install PyPDF2
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.1
    Uninstalling huggingface-hub-0.27.1:
      Successfully uninstalled huggingface-hub-0.27.1


In [5]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
import gradio as gr

ModuleNotFoundError: No module named 'gradio'

In [ ]:
# Extract text from PDF
pdf_path = "/content/HealthScore_FAQs.pdf"
reader = PdfReader(pdf_path)
faq_text = ""

for page in reader.pages:
    faq_text += page.extract_text()

#  extracted text to verify
print(f"Extracted Text: {faq_text[:729]}")

In [ ]:
# Split the text into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.create_documents([faq_text])

In [ ]:
# Embed the documents using HuggingFace embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(documents, embedding_model)

In [ ]:
#  Define a retrieval function
def retrieve_exact_question_answer(query):

    results = vector_store.similarity_search(query, k=5)

    for result in results:
        content = result.page_content
        lines = content.split("\n")

        for i, line in enumerate(lines):
            if query.lower() in line.lower():  # Match query with a question
                answer_lines = [line.strip()]  # Include the matched question
                for j in range(i + 1, len(lines)):
                    if lines[j].strip() == "" or lines[j].strip()[0].isdigit():  # Detect next question
                        break
                    answer_lines.append(lines[j].strip())
                return "\n".join(answer_lines)  # Return the question and full answer
    return "No exact match found. Please try rephrasing the question."


In [ ]:
test_query = "What causes fatigue and lack of energy?"
print("Testing query response:", retrieve_exact_question_answer(test_query))

In [ ]:
# Create the Gradio UI for user interaction
def chatbot_interface(user_query):
    if not user_query.strip():
        return "Please enter a valid question."

    try:
        answer = retrieve_exact_question_answer(user_query)
        return answer if answer else "No relevant answer found."
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [ ]:
# Launch the Gradio UI
ui = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about the document..."),
    outputs="text",
    title="PDF-Based Question Answering System",
    description="Type a question and get AI-generated answers based on the document.",
    allow_flagging="never"  # Disable the flagging option
)
# Run the Gradio app with a public shareable link
ui.launch(share=True)
